In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as spsp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
from PIL import Image
from scipy.integrate import ode

In [2]:
# Diffusion function
g = lambda s: 1/(1+s)
g_exp = lambda s: np.exp(-s)

# Test function
def f(x, alpha):
    first = 5-np.tanh(alpha*(x-1))-np.tanh(alpha*(x-2))
    second = np.tanh(alpha*(x-4)) + np.tanh(alpha*(x-5)) + 0.1*(np.sin(5*x))**2*np.sin(50*x)
    return first+second

# Create random 1D function
def generate_random1D(M):
    I = np.zeros(M+2)
    s = (M + 2)//5
    for i in range(5):
        I[i*s:(i+1)*s+1] = 10*np.random.randint(1, 5)
        
    # Add noise to interior points
    I[1:-1] = I[1:-1] + np.random.normal(0, 2, size = M)
    return I

In [3]:
# Diffusion function
g = lambda s: 1/(1+s)
g_exp = lambda s: np.exp(-s)

f = lambda x, y: x + y

def load_image( infilename , size = None) :
    img = Image.open( infilename )
    img.load()
    if size:
        img = img.resize(size)
    data = np.asarray( img, dtype="float32" )
    return data

# Display image given vector representation, and dimensions
def image_display(V, m, n):
    if len(V.shape) == 2:
        image = V.reshape((n, m, 3))
        return Image.fromarray(image.astype("uint8"), "RGB")

    else:
        image = V.reshape((n, m))
        return Image.fromarray(image.astype("uint8"))

def generate_squares2D(N, M):
    I = np.zeros((N+2, M+2))
    I[:N//2+1, :M//2+1] = 80
    I[:N//2+1, -(M//2+1):] = 190
    I[-(N//2+1):, :M//2+1] = 140
    I[-(N//2+1):, -(M//2+1):] = 230
    return I

def add_noise2D(I, scale = 10):
    N, M = I.shape
    N -= 2
    M -= 2
    I[1:-1, 1:-1] += np.random.randint(-scale, scale, size = (M,N))
    return I
    
def generate_random2D(M, N, scale = 10):
    I = generate_squares2D(N, M)
    add_noise2D(I)
    return I

In [4]:
def diffX(M, N, scheme = "central"):
    K = (M+2)*(N+2)
    Dx = np.zeros((K, K))
    dx = 1/(M+1)
    
    if scheme == "central":
        Bx = (-1 * np.eye(M+2, k = -1) + np.eye(M+2, k = 1))
        Bx[0, :3] = [-3, 4, -1]
        Bx[-1, -3:] = [1, -4, 3]
        Bx /= (2*dx)
        
    elif scheme == "forward":
        Bx = (-1 * np.eye(M+2) + np.eye(M+2, k = 1))
        Bx[-1, -2:] = [-1, 1]
        Bx /= dx
    else:
        raise Exception("Invalid scheme in Dx")    
        
    for i in range(N+2):
        Dx[i*(M+2):(i+1)*(M+2), i*(M+2):(i+1)*(M+2)] = Bx

    return Dx

def diffY(M, N, scheme = "central"):
    K = (M+2)*(N+2)
    Dy = np.zeros((K, K))
    dy = 1/(N+1)
    
    if scheme == "central":
        Dy = -np.eye(K, k = -(M+2)) + np.eye(K, k = M+2)
        Dy[:(M+2), :3*(M+2)] = np.hstack((-3*np.identity(M+2), 4*np.identity(M+2), -np.identity(M+2)))
        Dy[-(M+2):, -3*(M+2):] = np.hstack((np.identity(M+2), -4*np.identity(M+2), 3*np.identity(M+2)))
        Dy /= 2*dy
        
    elif scheme == "forward":
        Dy = -np.eye(K) + np.eye(K, k = M+2)
        Dy[:(M+2), :2*(M+2)] = np.hstack((-np.identity(M+2), np.identity(M+2)))
        Dy[-(M+2):, -2*(M+2):] = np.hstack((-np.identity(M+2), np.identity(M+2)))
        Dy /= dy
        
    else:
        raise Exception("Invalid scheme in Dy")
    
    return Dy  

def support_matrices_X(M, N):
    K = (M+2)*(N+2)
    
    Ξx = np.zeros((K, K))
    Ωx = np.zeros((K, K))
    Γx = np.zeros((K, K))
    
    #Block matrix for Ξx
    Xx = np.eye(M+2, k = -1) + np.eye(M+2)
    Xx[0, :2] = 0
    Xx[-1, -2:] = 0

    #Block matrix for Ωx
    Mx = -np.eye(M+2, k = -1) -2*np.eye(M+2) - np.eye(M+2, k = 1)
    Mx[0, :2] = 0
    Mx[-1, -2:] = 0

    #Block matrix for Γx
    Fx = np.eye(M+2) + np.eye(M+2, k = 1)
    Fx[0, :2] = 0
    Fx[-1, -2:] = 0
    
    # Fill in matrices
    for i in range(1, N+1):
        Ξx[i*(M+2):(i+1)*(M+2), i*(M+2):(i+1)*(M+2)] = Xx

    for i in range(1, N+1):
        Ωx[i*(M+2):(i+1)*(M+2), i*(M+2):(i+1)*(M+2)] = Mx

    for i in range(1, N+1):
        Γx[i*(M+2):(i+1)*(M+2), i*(M+2):(i+1)*(M+2)] = Fx

    return Ξx, Ωx, Γx

def support_matrices_Y(M, N):
    K = (M+2)*(N+2)
    Ξy = np.zeros((K, K))
    Ωy = np.zeros((K, K))
    Γy = np.zeros((K, K))
    
    for i in range(1, N+1):
        Ξy[i*(M+2)+1:(i+1)*(M+2)-1, (i-1)*(M+2)+1:i*(M+2)-1] = np.identity(M)
        Ξy[i*(M+2)+1:(i+1)*(M+2)-1, i*(M+2)+1:(i+1)*(M+2)-1] = np.identity(M)

    for i in range(1, N+1):
        Ωy[i*(M+2)+1:(i+1)*(M+2)-1, (i-1)*(M+2)+1:i*(M+2)-1] = -np.identity(M)
        Ωy[i*(M+2)+1:(i+1)*(M+2)-1, i*(M+2)+1:(i+1)*(M+2)-1] = -2*np.identity(M)
        Ωy[i*(M+2)+1:(i+1)*(M+2)-1, (i+1)*(M+2)+1:(i+2)*(M+2)-1] = -np.identity(M)

    for i in range(1, N+1):
        Γy[i*(M+2)+1:(i+1)*(M+2)-1, i*(M+2)+1:(i+1)*(M+2)-1] = np.identity(M)
        Γy[i*(M+2)+1:(i+1)*(M+2)-1, (i+1)*(M+2)+1:(i+2)*(M+2)-1] = np.identity(M)
    
    return Ξy, Ωy, Γy

def assemble_A(u, M, N, g, Dx, Dy, Ξx, Ωx, Γx, Ξy, Ωy, Γy):
    dx = 1/(M+1)
    dy = 1/(N+1)
    G = g(Dx.dot(u)**2 + Dy.dot(u)**2)
    
    ξx = Ξx.dot(G)
    ωx = Ωx.dot(G)
    γx = Γx.dot(G)

    ξy = Ξy.dot(G)
    ωy = Ωy.dot(G)
    γy = Γy.dot(G)
    
    x_diags = (ξx[1:], ωx, γx[:-1])
    y_diags = (ξy[(M+2):], ωy, γy[:-(M+2)])
    
    Ax = spsp.diags(x_diags, (-1, 0, 1))
    Ay = spsp.diags(y_diags, (-(M+2), 0, M+2))
    A = 1/2*(Ax/dx**2 + Ay/dy**2)
    return A


def iteration_echo(M, N, G, A, Dy, Dx, u):
    K = (M+2)*(N+2)
    plt.figure(figsize=(12, 8))
    plt.subplot(131)
    plt.imshow(u.reshape(N+2, M+2), cmap = "gray", vmin = 0, vmax = 255)
    
    plt.subplot(132)
    plt.imshow((Dy.dot(u)**2 + Dx.dot(u)**2).reshape(N+2, M+2), cmap = "gray")
    
    plt.subplot(133)
    plt.imshow(G.reshape(N+2, M+2), cmap = "gray", vmax = 5e-2)

    plt.show()


def solve_FE(u0, g, M, N, T, dt):
    dx = 1/(M+1)
    dy = 1/(N+1)
    r = 1/2 * (dt/(dx**2 + dy**2))
    
    U = np.zeros((T, K))
    U[0] = u0   
    
    Dx = diffX(M, N)
    Dy = diffY(M, N)
    
    Ξx, Ωx, Γx = support_matrices_X(M, N)
    Ξy, Ωy, Γy = support_matrices_Y(M, N)
    
    for it in range(T-1):
        A = assemble_A(U[it], M, N, g, Dx, Dy, Ξx, Ωx, Γx, Ξy, Ωy, Γy)
        G = g(Dx.dot(U[it])**2 + Dy.dot(U[it])**2)
        U[it+1] = U[it]  + dt * A.dot(U[it])
        
        if it % (T//10) == 0:
            iteration_echo(M, N, G, A, Dy, Dx, U[it])            
    return U


def solve_BE(u0, g, M, N, T, dt):
    dx = 1/(M+1)
    dy = 1/(N+1)    

    U = np.zeros((T, K))
    U[0] = u0   
    
    Dx = diffX(M, N)
    Dy = diffY(M, N)
    
    Ξx, Ωx, Γx = support_matrices_X(M, N)
    Ξy, Ωy, Γy = support_matrices_Y(M, N)

    for it in range(T-1):
        A = assemble_A(U[it], M, N, g, Dx, Dy, Ξx, Ωx, Γx, Ξy, Ωy, Γy)
        G = g(Dx.dot(U[it])**2 + Dy.dot(U[it])**2)
        
        U[it+1] = spla.spsolve(spsp.identity((M+2)*(N+2)) - dt * A, U[it])
        
        if it % (T//100) == 0:
            iteration_echo(M, N, G, A, Dy, Dx, U[it])            
    return U


# Solve equation using semi-implicit 
def solve_CN(u0, g, M, N, T, dt):
    dx = 1/(M+1)
    dy = 1/(N+1)
    r = 1/2 * (dt/(dx**2 + dy**2))
    

    U = np.zeros((T, K))
    U[0] = u0   
    
    Dx = diffX(M, N)
    Dy = diffY(M, N)
    
    Ξx, Ωx, Γx = support_matrices_X(M, N)
    Ξy, Ωy, Γy = support_matrices_Y(M, N)

    for it in range(T-1):
        A = assemble_A(U[it], M, N, g, Dx, Dy, Ξx, Ωx, Γx, Ξy, Ωy, Γy)
        G = g(Dx.dot(U[it])**2 + Dy.dot(U[it])**2)
        
        U[it+1] = spla.spsolve(spsp.identity((M+2)*(N+2)) - dt/2 * A, (spsp.identity((M+2)*(N+2)) + dt/2 *A).dot(U[it]))
        
        if it % (T//100) == 0:
            iteration_echo(M, N, G, A, Dy, Dx, U[it])            

    return U

def solve_RGB_BE(u0, g, M, N, T, dt):
    dx = 1/(M+1)
    dy = 1/(N+1)
    

    U = np.zeros((T, K, 3))
    U[0] = u0
    
    Dx = diffX(M, N)
    Dy = diffY(M, N)
    
    Ξx, Ωx, Γx = support_matrices_X(M, N)
    Ξy, Ωy, Γy = support_matrices_Y(M, N)

    for it in range(T-1):
        print(it)
        Ar = assemble_A(U[it,:, 0], M, N, g, Dx, Dy, Ξx, Ωx, Γx, Ξy, Ωy, Γy)
        Ag = assemble_A(U[it,:, 1], M, N, g, Dx, Dy, Ξx, Ωx, Γx, Ξy, Ωy, Γy)
        Ab = assemble_A(U[it,:, 2], M, N, g, Dx, Dy, Ξx, Ωx, Γx, Ξy, Ωy, Γy)
        
        U[it+1,:, 0] = spla.spsolve(spsp.identity((M+2)*(N+2)) - dt * Ar, U[it, :, 0])
        U[it+1,:, 1] = spla.spsolve(spsp.identity((M+2)*(N+2)) - dt * Ag, U[it, :, 1])
        U[it+1,:, 2] = spla.spsolve(spsp.identity((M+2)*(N+2)) - dt * Ab, U[it, :, 2])

        if it % (T//10) == 0:
            plt.figure(figsize = (8,8))
            im = image_display(U[it], N+2, M+2)
            plt.imshow(im)
            plt.show()
#             iteration_echo(M, N, G, A, Dy, Dx, U[it])            
    return U

In [5]:
g1 = lambda s: x*np.ones(x.size)
g2 = lambda s: 1/(1+s)

x = np.linspace(0, 6, 101)

g2(x)

array([1.        , 0.94339623, 0.89285714, 0.84745763, 0.80645161,
       0.76923077, 0.73529412, 0.70422535, 0.67567568, 0.64935065,
       0.625     , 0.60240964, 0.58139535, 0.56179775, 0.54347826,
       0.52631579, 0.51020408, 0.4950495 , 0.48076923, 0.46728972,
       0.45454545, 0.44247788, 0.43103448, 0.42016807, 0.40983607,
       0.4       , 0.390625  , 0.38167939, 0.37313433, 0.3649635 ,
       0.35714286, 0.34965035, 0.34246575, 0.33557047, 0.32894737,
       0.32258065, 0.3164557 , 0.31055901, 0.30487805, 0.2994012 ,
       0.29411765, 0.28901734, 0.28409091, 0.27932961, 0.27472527,
       0.27027027, 0.26595745, 0.2617801 , 0.25773196, 0.25380711,
       0.25      , 0.24630542, 0.24271845, 0.23923445, 0.23584906,
       0.23255814, 0.2293578 , 0.22624434, 0.22321429, 0.22026432,
       0.2173913 , 0.21459227, 0.21186441, 0.20920502, 0.20661157,
       0.20408163, 0.2016129 , 0.19920319, 0.19685039, 0.19455253,
       0.19230769, 0.19011407, 0.18796992, 0.18587361, 0.18382

In [6]:
# Spatial discretization
M = 100
N = 100

M -= 2
N -= 2
K = (M+2) * (N+2)
dx = 1/(M+1)
dy = 1/(N+1)

# Initial conditions
# Number of iterations, and timestep
T = 10
dt = 1e-3

# g = lambda s: 1/(1+s)
g1 = lambda s: s*np.ones(s.size)

# I = generate_random2D(N, M, scale = 2)
I = load_image("lena-64x64.jpg", (M+2, N+2))
U = np.zeros((T, K, 3))
U[0] = I.reshape(K, 3)
plt.figure(figsize = (12, 8))
im = image_display(U[0], M+2, N+2)
plt.imshow(im)
plt.show()
# U = solve_RGB_BE(I.reshape(K,3), g, M, N, T, dt )

ValueError: cannot reshape array of size 10000 into shape (10000,3)

In [ ]:
g(U[0]**2 + 2*U[0]**2).shape

In [ ]:
# Spatial discretization
M = 30
N = 30

K = (M+2) * (N+2)
dx = 1/(M+1)
dy = 1/(N+1)

# Initial conditions
# Number of iterations, and timestep
T = 100000
dt = 1e-6

# g = lambda s: 1
g = lambda s: 1/(1+s)

I = generate_random2D(N, M, scale = 2)
# I = load_image("lena-64x64.jpg", (M+2, N+2))
I = add_noise2D(I)
U = np.zeros((T, K))
U[0] = I.reshape(K)
solve_FE(I.reshape(K), g, M, N, T, dt)

In [ ]:
# U = solve_BE(U[0], g, M, N, T, dt )

In [7]:
# image_display(U[0], N+2, M+2)
U = solve_RGB_BE(I.reshape(K,3), g, M, N, T, dt )

ValueError: cannot reshape array of size 10000 into shape (10000,3)